Данный notebook будет опираться на следующую статью: [тык](https://habr.com/ru/company/avito/blog/709596/). Здесь мы поэтапно разберем и разжуем все изложенные в статье кейсы. Иногда будем это делать с доказательствами или давать ссылки на них.

## Эксперимент 1

Идея такая: попробуем показать на примере, что тест Манна-Уитни не проверяет равенство средних. Давайте для этого возьмем выборки из двух распределений, у которых равны средние и медианы, а также они будут симметричны относительно нуля. После этого проведем A/A тест на уровне значимости $\alpha = 0.05$ на примере двух тестов - тест Манна-Уитни и Т-тест, где за нулевую гипотезу возьмем равенство средних двух распределений. А также потроим доверительный интервал для истинной ошибки 1-ого рода и поясним, как мы это сделали.

In [11]:
import pandas as pd
from tqdm.notebook import tqdm # tqdm – библиотека для визуализации прогресса в цикле
import numpy as np
import scipy.stats as sps
from statsmodels.stats.proportion import proportion_confint

In [12]:
def experiment_1(length=5000, lb_1=-5, ub_1=5, lb_2=-150, ub_2=150, size_1=1500, size_2=1500):
    ttest_effect_counter = 0
    mu_effect_counter = 0

    ttest_pvalues = []
    mu_pvalues = []

    for i in tqdm(range(length)):
        uniform_1 = np.random.uniform(low=lb_1, high=ub_1, size=size_1)
        uniform_2 = np.random.uniform(low=lb_2, high=ub_2, size=size_2)
        ttest_pvalues.append(sps.ttest_ind(a=uniform_1, b=uniform_2, equal_var=False, alternative='two-sided').pvalue)
        mu_pvalues.append(sps.mannwhitneyu(x=uniform_1, y=uniform_2, alternative='two-sided').pvalue)

        if ttest_pvalues[i] < 0.05:
            ttest_effect_counter += 1
        if mu_pvalues[i] < 0.05:
            mu_effect_counter += 1

    p_1 = ttest_effect_counter / length
    p_2 = mu_effect_counter / length
    conf_p_1 = proportion_confint(count=ttest_effect_counter, nobs=length, alpha=0.05, method='wilson')
    conf_p_2 = proportion_confint(count=mu_effect_counter, nobs=length, alpha=0.05, method='wilson')

    print(f"Доля полученных эффектов для ttest = {p_1}\nДоверительный интервал для этой доли = {conf_p_1}",
        f"Доля полученных эффектов для mutest = {p_2}\nДоверительный интервал для этой доли = {conf_p_2}", sep='\n\n')

In [13]:
experiment_1()

  0%|          | 0/5000 [00:00<?, ?it/s]

Доля полученных эффектов для ttest = 0.0516
Доверительный интервал для этой доли = (0.045805193052696626, 0.05808328205163061)

Доля полученных эффектов для mutest = 0.115
Доверительный интервал для этой доли = (0.10645134542601838, 0.12413978507167563)


Как мы видим, доля полученных эффектов там, где их быть не должно, - для Т-тест близка к 0.05, что согласуется с тем, что p-value равномерно распределена при верной нулевой гипотезе. Поэтому если мы отвергали основную гипотезу на уровне $\alpha=0.05$, то и доля должна быть близка к этому значению $\alpha=0.05$. Как мы видим, для Манна-Уитни теста это далеко не так, значит, с необходимостью либо нулевая гипотеза не может быть верна, либо этот тест вообще не подразумевал в нулевой гипотезе то, что мы пытаемся проверить! Так как первый вариант невозможен (ведь средние и правда равны в двух выборках), значит, остается только второй вариант.

Мы строили доверительный интервал для истинной ошибки 1-ого рода с помощью метода Вилсона. Если коротко, это просто нормальная аппроксимация для классической t-статистики. Где в качестве выборочного среднего берется доля, полученная в нашем эксперименте. Как несложно показать, данная t-статистика имеет нормальное распределение (Теорема Слуцкого + ЗБЧ + Теорема непрерывности). Стало быть, легко построить ассимптотический доверительный интервал, что в этом методе и делается. Подробнее можно почитать вот [здесь](https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval)

На самом деле, тест Манна-Уитни проверяет полное совпадение распределений у двух групп: тестовой и контрольной, против сложной альтернативы, что распределения отличаются на некоторый сдвиг $C$!

К слову, было логично получить долю ложных прокрасов аж в два раза больше, чем у Т-теста. Так как Манна-Уитни проверяет более сильный факт.

Для этого снова запустим нашу функцию `experiment_1`, но уже подадим туда две выборки, взятые из **одинакового** распределения, и посмотрим, что из этого получится:)

In [14]:
experiment_1(length=5000, lb_1=-5, ub_1=5, lb_2=-5, ub_2=5, size_1=2500, size_2=2500)

  0%|          | 0/5000 [00:00<?, ?it/s]

Доля полученных эффектов для ttest = 0.054
Доверительный интервал для этой доли = (0.048070671178201296, 0.06061411895678967)

Доля полученных эффектов для mutest = 0.0544
Доверительный интервал для этой доли = (0.04844859105944088, 0.06103558491399404)


Да! 

Мы как раз получили обе близкие к $\alpha = 0.05$ доли, как и должны были получить теоретически.

## Эксперимент 2